In [1]:
!pip install selenium
!pip install webdriver_manager
!pip install pyperclip

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pyperclip

import os
import time
from datetime import timedelta

In [5]:
import pandas as pd

# 빈 데이터프레임 만들기
# data = pd.DataFrame({'행정동명':[], '연월':[], '유입인구':[], '유입인구_남성':[], '유입인구_여성':[]})
data = pd.DataFrame({'행정동명':[], '연월':[], '유입인구':[]})

data

,행정동명,연월,유입인구


In [7]:
# 셀레니움 드라이버 설정
options = webdriver.ChromeOptions()
#user_data_dir = 'C:/Users/SSW/AppData/Local/Google/Chrome/User Data'  # 사용자 프로필 경로
#profile_name = 'Profile 3'
#options.add_argument(f"user-data-dir={user_data_dir}") # 사용자 프로필 경로 지정
#options.add_argument(f"profile-directory={profile_name}") # 사용자 프로필 지정
#options.add_argument("remote-debugging-port=9222")  # 디버깅 포트 설정

options.add_experimental_option('prefs', {
#   "download.default_directory": "E:/python/prgrms-project4/od_data/ilban_bus",  # 다운로드 경로 설정(현재는 오류로 막아둠)
    "download.prompt_for_download": False,              # 다운로드 창 자동 처리
    "directory_upgrade": True,
})
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://data.seoul.go.kr/livPopu/html/dashboard.html')

In [9]:
class WebScraper:
    # WebScraper 클래스 정의
    def __init__(self):
        # new_row 초기화
        # self.new_row = {'행정동명': [], '연월': [], '유입인구': [], '유입인구_남성': [], '유입인구_여성': []}
        self.new_row = {'행정동명': [], '연월': [], '유입인구': []}
    
    def set_condition(self, driver, code_gu, code_dong, code_yrmnt):
        # 조건(보기상태/조회지역/년월/주차) 입력 및 행정동, 연월을 new_row에 입력하는 메서드
        # 자치구 코드(code_gu)는 2(강남구)-26(중랑구),
        # 행정동 코드(code_dong)는 유동적이나 2-28(송파)까지 존재
        # 년월 코드는 4(2024년 7월)-35(2022년 1월)까지 역순으로 존재
        
        # 로딩화면 사라질 때까지 대기
        WebDriverWait(driver, 20).until(
                EC.invisibility_of_element_located((By.XPATH, '//*[@id="spr-010"]/div[2]'))
            )
        
        # 옵션 선택 버튼 누르기
        WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="wrapper"]/div/div[3]/a'))
            )
        driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[3]/a').click()
        
        # 보기상태(유입인구) 클릭
        driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[3]/div/div[2]/form/div[1]/div[2]/select/option[2]').click()
        time.sleep(0.2)

        # 자치구 선택
        gu = driver.find_element(By.XPATH, f'//*[@id="wrapper"]/div/div[3]/div/div[2]/form/div[2]/div[2]/select[1]/option[{code_gu}]')
        gu.click()

        # 행정동 선택
        try:
            dong = driver.find_element(By.XPATH, f'//*[@id="wrapper"]/div/div[3]/div/div[2]/form/div[2]/div[2]/select[2]/option[{code_dong}]')
            dong.click()
            time.sleep(0.2)

        except NoSuchElementException:
            print(f"Error: 구 코드 {code_gu}의 동 코드 {code_dong}가 존재하지 않습니다.")
            return False  # 동 코드가 없으면 False 반환하여 다음 구 코드로 넘어감
        
        # 년월 선택
        yrmnt = driver.find_element(By.XPATH, f'//*[@id="wrapper"]/div/div[3]/div/div[2]/form/div[4]/div[2]/select/option[{code_yrmnt}]')
        yrmnt.click()

        # 조회버튼 클릭하여 조건 입력 완료
        WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="wrapper"]/div/div[3]/div/div[2]/div/a'))
            )
        driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[3]/div/div[2]/div/a').click()
        time.sleep(0.2)

        # 선택한 조건 저장
        self.new_row['행정동명'] = f'{gu.text} {dong.text}'
        self.new_row['연월'] = yrmnt.text

        return self.new_row

    def scrap_pop_total(self, driver):
        try:
        # 유입인구 불러오기
        # 로딩화면 사라질 때까지 대기
            WebDriverWait(driver, 20).until(
                EC.invisibility_of_element_located((By.XPATH, '//*[@id="spr-010"]/div[2]'))
            )
    
            # 유입인구 요소 호출
            pop_elem = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[1]/div[3]/div[2]/div[2]/div[1]/div[2]')
            
            # flex-item의 동적 로딩 문제로 인해 값의 업데이트가 늦어 요소 직접 호출 대신 복사+붙여넣기 actionchains로 대체
            # ActionChains로 더블클릭해서 텍스트 선택
            actions = ActionChains(driver)
            actions.double_click(pop_elem).perform()  # 더블클릭으로 텍스트 선택
    
            # Ctrl + C를 눌러 클립보드에 복사
            actions.key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
    
            # 클립보드에서 텍스트 가져오기
            pop_total = pyperclip.paste()  # 클립보드에서 복사된 텍스트를 가져옴  
            
            # 유입인구 값을 new_row에 저장
            self.new_row['유입인구'] = pop_total
    
            return self.new_row
    
        except NoSuchElementException:
            # 웹 요소를 찾을 수 없으면 처리
            print(f"Error: 유입인구 정보를 찾는 데 실패했습니다. {e}")
            return None
    
        except pyperclip.PyperclipException as e:
            # 클립보드 작업에서 오류가 발생한 경우
            print(f"Error: 클립보드에서 텍스트를 가져오는 데 실패했습니다. {e}")
            return None
    
        except Exception as e:
            # 기타 모든 예외 처리
            print(f"Unexpected error: {e}")
            return None

    '''
    # 동별 유입인구 테이블상에서 상위 100개 출발지까지밖에 조회되지 않으므로 아래 메서드는 포기
    def scrap_pop_gender(self, driver):
        # 더보기 버튼 클릭
        download_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="table-01"]/tfoot/tr/td/a'))
            )
        # JavaScript로 클릭 강제
        driver.execute_script("arguments[0].click();", download_button)
       
        WebDriverWait(driver, 20).until(
           EC.presence_of_all_elements_located((By.XPATH, '//*[@id="table-01"]/tbody/tr/td[3]'))
           )

        # 해당 열만 불러오기
        # pop_male_elem = driver.find_elements(By.XPATH, '//*[@id="table-01"]/tbody/tr/td[3]/span[1]')
        # pop_female_elem = driver.find_elements(By.XPATH, '//*[@id="table-01"]/tbody/tr/td[3]/span[2]')
        
        # 성별로 출발지별 유입인구 값들을 한 번에 더하기
        
        pop_male = sum(
            map(
                lambda x: int(x.get_attribute('textContent').replace(',', '')) 
                if x.get_attribute('textContent').replace(',', '').isdigit() 
                else 0, 
                pop_male_elem
            )
        )    
        pop_female = sum(
            map(
                lambda x: int(x.get_attribute('textContent').replace(',', '')) 
                if x.get_attribute('textContent').replace(',', '').isdigit() 
                else 0, 
                pop_female_elem
            )
        )

        # new_row에 성별 유입인구 추가
        self.new_row['유입인구_남성'] = pop_male
        self.new_row['유입인구_여성'] = pop_female

        return self.new_row
    '''
    def print_row(self):
        # 저장된 new_row 출력
        print(self.new_row)
    
    def add_row(self, data):
        # 저장된 new_row를 데이터프레임에 행으로 추가
        data.loc[len(data)] = self.new_row

In [ ]:
'''
# 테스트
scraper = WebScraper()
scraper.set_condition(driver, 3, 3, 10)
scraper.scrap_pop_total(driver)
# scraper.scrap_pop_gender(driver)
# scraper.print_row()
scraper.add_row(data)
data
'''

In [ ]:
scraper = WebScraper()

# 스크래핑 시작 시간 기록
start_time = time.time()

# 자치구 코드 범위 (2~26)
for code_gu in range(2, 27):  
    # 행정동 코드 범위 (2~28) - 행정동이 없을 시 중단해야 함
    for code_dong in range(2, 29):
        # 년월 코드 범위 (4~35, 역순)
        for code_yrmnt in range(4, 36):
            # 조건 설정 및 행정동과 연월 선택
            condition = scraper.set_condition(driver, code_gu, code_dong, code_yrmnt)

            # 행정동이 없으면 다음 자치구로 넘어감
            if condition is False:
                break  # 현재 자치구에서 행정동이 없으므로 더 이상 진행하지 않고 다음 자치구로 넘어감
            
            # 유입인구 스크래핑 시도
            pop_total = scraper.scrap_pop_total(driver)
            
            if pop_total is not None:
                # 정상적으로 데이터를 가져오면 행 추가
                scraper.add_row(data)
            else:
                print(f"유입인구 데이터를 불러오지 못했습니다. 자치구 코드 {code_gu}, 행정동 코드 {code_dong}, 년월 코드 {code_yrmnt}.")

# 웹드라이버 종료
driver.quit()

# 스크래핑 종료 시간 기록 및 총 소요 시간 계산
end_time = time.time()
total_time = end_time - start_time

# timedelta로 자동 변환
formatted_time = str(timedelta(seconds=total_time))

# 시간, 분, 초 단위로 출력
print(f"스크래핑 총 소요 시간: {formatted_time}")

# 유입인구 dtype정수형으로 변경
data['유입인구'] = data['유입인구'].str.replace(',', '').astype(int)

# 수집 결과 csv로 저장
data.to_csv('pop.csv', index=False, encoding='utf-8')

In [ ]:
data.info()

In [ ]:
data.head(6)